# ***🎌 Anime Recommendation System 🎌***
# ---
## 🏮 ***Find Your Next Favorite Anime!*** 🏮
### Welcome to our **Anime Recommendation System**! This system helps you discover similar animes based on user ratings. Whether you're looking for your next binge-worthy series or exploring new genres, we've got you covered! 🍿✨

## 🌸 **Background & Motivation**
Anime is a vast and diverse entertainment medium with thousands of titles across various genres. Finding similar anime based on personal preferences can be challenging. Our system uses **collaborative filtering** to recommend anime based on user ratings, making it easier to discover hidden gems. 🎥🎭

---

### ***Importing Libraries***

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from IPython.display import display, HTML

### 📜 ***Loading Anime Dataset***

In [13]:
anime = pd.read_csv('data/anime.csv')
anime.head(2)

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665


### ⭐ ***Loading Ratings Dataset***

In [14]:
rating = pd.read_csv('data/rating.csv')
rating.head(2)

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1


### 🔍 ***Dataset Overview***

In [15]:
anime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  object 
 2   genre     12232 non-null  object 
 3   type      12269 non-null  object 
 4   episodes  12294 non-null  object 
 5   rating    12064 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 672.5+ KB


In [16]:
rating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7813737 entries, 0 to 7813736
Data columns (total 3 columns):
 #   Column    Dtype
---  ------    -----
 0   user_id   int64
 1   anime_id  int64
 2   rating    int64
dtypes: int64(3)
memory usage: 178.8 MB


### 🔄 ***Renaming Columns***

In [17]:
anime.rename(columns={"rating":"anime_rating"} , inplace = True)
rating.rename(columns={"rating":"user_rating"} , inplace = True)

### 🔗 ***Merging Datasets***

In [18]:
total_df = rating.merge(anime , on = 'anime_id')
total_df.head()

,user_id,anime_id,user_rating,name,genre,type,episodes,anime_rating,members
0,1,20,-1,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,7.81,683297
1,1,24,-1,School Rumble,"Comedy, Romance, School, Shounen",TV,26,8.06,178553
2,1,79,-1,Shuffle!,"Comedy, Drama, Ecchi, Fantasy, Harem, Magic, R...",TV,24,7.31,158772
3,1,226,-1,Elfen Lied,"Action, Drama, Horror, Psychological, Romance,...",TV,13,7.85,623511
4,1,241,-1,Girls Bravo: First Season,"Comedy, Ecchi, Fantasy, Harem, Romance, School",TV,11,6.69,84395


### 🛠️ ***Data Cleaning***

In [19]:
total_df = total_df[total_df['user_rating'] != -1]
total_df.dropna(inplace=True)
total_df.shape

(6337146, 9)

### 📊 ***Filtering Active Users***

In [20]:
user_counts = total_df['user_id'].value_counts()
filtered_users = user_counts[user_counts > 200].index
filtered_users

Index([42635, 53698, 57620, 59643, 51693, 45659,  7345, 12431, 65840, 22434,
       ...
       29140, 58808, 14619, 52795, 45506, 34632, 11243, 28800, 39161, 61408],
      dtype='int64', name='user_id', length=8638)

In [21]:
total_df = total_df[total_df['user_id'].isin(filtered_users)]
total_df.drop_duplicates(['user_id', 'anime_id'], inplace=True)
total_df.shape

(3164693, 9)

In [22]:
total_df.isna().sum()

user_id         0
anime_id        0
user_rating     0
name            0
genre           0
type            0
episodes        0
anime_rating    0
members         0
dtype: int64

### 🗂️ ***Creating Pivot Table***

In [24]:
anime_pivot = total_df.pivot_table(columns='user_id' , index = 'name' , values='user_rating')
anime_pivot

user_id,5,7,17,38,43,46,123,129,139,160,...,73406,73417,73422,73457,73460,73476,73499,73502,73503,73507
name,,,,,,,,,,,,,,,,,,,,,
&quot;0&quot;,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
&quot;Bungaku Shoujo&quot; Kyou no Oyatsu: Hatsukoi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN
&quot;Bungaku Shoujo&quot; Memoire,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
&quot;Bungaku Shoujo&quot; Movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN
&quot;Eiji&quot;,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
xxxHOLiC Kei,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,8.0,...,10.0,NaN,NaN,NaN,NaN,NaN,NaN,10.0,8.0,9.0
xxxHOLiC Movie: Manatsu no Yoru no Yume,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9.0,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,9.0
xxxHOLiC Rou,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,...,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN


In [25]:
anime_pivot.shape

(9785, 8638)

In [26]:
anime_pivot.fillna(0 , inplace = True)
anime_pivot

user_id,5,7,17,38,43,46,123,129,139,160,...,73406,73417,73422,73457,73460,73476,73499,73502,73503,73507
name,,,,,,,,,,,,,,,,,,,,,
&quot;0&quot;,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
&quot;Bungaku Shoujo&quot; Kyou no Oyatsu: Hatsukoi,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0
&quot;Bungaku Shoujo&quot; Memoire,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
&quot;Bungaku Shoujo&quot; Movie,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0
&quot;Eiji&quot;,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
xxxHOLiC Kei,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,8.0,...,10.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,8.0,9.0
xxxHOLiC Movie: Manatsu no Yoru no Yume,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,9.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,9.0
xxxHOLiC Rou,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,...,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0


### 🧠 ***Training Recommendation Model***


In [28]:
anime_sparse = csr_matrix(anime_pivot)
model = NearestNeighbors(algorithm='brute')
model.fit(anime_sparse)

NearestNeighbors(algorithm='brute')

In [29]:
distance, suggestion = model.kneighbors(anime_pivot.iloc[100,:].values.reshape(1,-1), n_neighbors=6 )
distance

array([[0.        , 2.        , 2.23606798, 2.23606798, 2.23606798,
        2.23606798]])

In [30]:
for i in range(len(suggestion)):
    print(anime_pivot.index[suggestion[i]])

Index(['AWOL', 'Circuit no Ookami II: Modena no Ken',
       'Bishoujo Animerama: Manami no Michi Tono Sou Nyuu!?',
       'Bishoujo Animerama: Minami no Watashi no Heart ni Tacchi Shite...',
       '4.Eyes', 'Bara Bara Film'],
      dtype='object', name='name')


### 🔎 ***Anime Recommendation Function***

In [31]:
def recommend_anime(anime_name):
    if anime_name not in anime_pivot.index:
        print(f"❌ Anime '{anime_name}' not found in dataset.")
        return
    
    anime_id = np.where(anime_pivot.index == anime_name)[0][0]
    distance, suggestions = model.kneighbors(anime_pivot.iloc[anime_id, :].values.reshape(1, -1), n_neighbors=6)
    
    print(f"🔎 You searched for: '{anime_name}'")
    print("Recommended animes:")
    for i in suggestions[0]:
        suggested_anime = anime_pivot.index[i]
        if suggested_anime != anime_name:
            print(f"✅ {suggested_anime}")

### 🎯 ***Example Usage***

In [32]:
anime_name = "4.Eyes"
recommend_anime(anime_name)

🔎 You searched for: '4.Eyes'
Recommended animes:
✅ Memories: Younenki no Joukei
✅ Goldfish Fetish
✅ Kaze no Kokyuu: Animation ni Yoru Oufuku Shokan
✅ Jinkou no Rakuen
✅ Landscape
